## Imports Required

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

## Class

In [ ]:
class FlightTicket:
    def __init__(self):
        self.parameter_template = {
            "price_coach_low": { "dtype": float, "prompt": "Coach lower ticket price"},
            "price_coach_high": { "dtype": float, "prompt": "Coach higher ticket price"},
            "price_fclass_low": { "dtype": float, "prompt": "First class lower ticket price"},
            "price_fclass_high": { "dtype": float, "prompt": "First class lower ticket price"},
            "booking_proba_coach_lowpr": { "dtype": float, "prompt": "Probability of booking for coach at lower price"},
            "booking_proba_coach_highpr": { "dtype": float, "prompt": "Probability of booking for coach at higher price"},
            "booking_proba_fclass_lowpr": { "dtype": float, "prompt": "Probability of booking for first class at lower price"},
            "booking_proba_fclass_highpr": { "dtype": float, "prompt": "Probability of booking for first class at higher price"},
            "prob_change_on_fclass_sold": { "dtype": float, "prompt": "Change in coach probability when first class is sold out"},
            "cost_of_bumping": { "dtype": float, "prompt": "Cost of bumping passenger to first class"},
            "cost_of_stipend": { "dtype": float, "prompt": "Cost of bumping passenger off the plane"},
            "discount": { "dtype": float, "prompt": "Annual Discount rate (as decimal)"},
            "proba_coach_showing_up": { "dtype": float, "prompt": "Probability of coach passenger showing up"},
            "proba_fclass_showing_up": { "dtype": float, "prompt": "Probability of first class passenger showing up"},
            "n_seats_coach": { "dtype": int, "prompt": "Number of seats in coach"},
            "n_seats_fclass": { "dtype": int, "prompt": "Number of seats in first class"},
            "overbook_limit": { "dtype": int, "prompt": "Allowed overbooking limit"},
            "days_left": { "dtype": int, "prompt": "Days left until departure"}
        }

        self.solved = False
        self.U = None
        self.V = None

    def get_parameter_template(self):
        return self.parameter_template
    
    def set_parameters(self, param: dict):
        for key in param.keys():
            assert key in self.parameter_template.keys()

        for key in self.parameter_template.keys():
            assert key in param.keys()
        
        for key in param.keys():
            setattr(self, key, param[key])

        self.price_coach = [self.price_coach_low, self.price_coach_high]
        self.price_fclass = [self.price_fclass_low, self.price_fclass_high]

        self.prob_coach = [self.booking_proba_coach_lowpr, self.booking_proba_coach_highpr]
        self.prob_fclass = [self.booking_proba_fclass_lowpr, self.booking_proba_fclass_highpr]
        self.prob_coach_fsold = [x + self.prob_change_on_fclass_sold for x in self.prob_coach]
        
        self.delta = 1/(1+(self.discount/365))
        self.seats_coach_eff = self.n_seats_coach + self.overbook_limit

        self.s1N = self.n_seats_fclass+1
        self.s2N = self.seats_coach_eff+1
        self.tN = self.days_left+1

        self.V = np.zeros((self.s1N, self.s2N, self.tN))
        self.U = np.array([[['#00000080']*self.tN]*self.s2N]*self.s1N)

    def calculate_initial_conditions(self):
        for s1 in range(self.s1N):
            for s2 in range(self.s2N):
                self.V[s1, s2, self.tN-1] = -self.__expected_cost__(self.n_seats_fclass-s1, self.seats_coach_eff-s2)

    def calculate_values(self):
        for t in reversed(range(self.tN-1)):
            for s1 in range(self.s1N):
                for s2 in range(self.s2N):
                    if s1 == 0 and s2 == 0:
                        self.V[s1, s2, t] = self.delta*self.V[s1, s2, t+1]
                    else:
                        prob_coach_eff = self.prob_coach if s1 != 0 else self.prob_coach_fsold
                        prob_fclass_eff = self.prob_fclass if s1 != 0 else [0, 0]
                        prob_coach_eff = prob_coach_eff if s2 != 0 else [0, 0]
                        
                        values = list()
                        for x1 in range(1,-1,-1):
                            for x2 in range(1, -1, -1):
                                value = self.price_coach[x1]*prob_coach_eff[x1] + self.price_fclass[x2]*prob_fclass_eff[x2] + self.delta*(
                                    prob_coach_eff[x1]*prob_fclass_eff[x2]*self.V[max(s1-1, 0), max(s2-1,0), t+1] + \
                                    prob_coach_eff[x1]*(1-prob_fclass_eff[x2])*self.V[s1, max(s2-1,0), t+1] + \
                                    (1-prob_coach_eff[x1])*prob_fclass_eff[x2]*self.V[max(s1-1, 0), s2, t+1] + \
                                    (1-prob_coach_eff[x1])*(1-prob_fclass_eff[x2])*self.V[s1, s2, t+1]
                                )
                                values.append(value)

                        # values[0] -> Coach High,  Fclass High
                        # values[1] -> Coach High,  Fclass Low
                        # values[2] -> Coach Low,   Fclass High
                        # values[3] -> Coach Low,   Fclass Low

                        self.V[s1, s2, t] = max(values)

                        tmp = np.argmax(values)
                        rgb = [0,0,0]
                        # Set the green pixel intensity based on Fclass price - 255 for high price, 127 for low price
                        if s1:
                            rgb[1] = ((int(tmp == 0 or tmp == 2) + 1)*128)-1
                        
                        # Set the blue pixel intensity based on coach price - 255 for high price, 127 for low price
                        if s2:
                            rgb[2] = ((int(tmp == 0 or tmp == 1) + 1)*128)-1

                        self.U[s1, s2, t] = self.__rgb_to_hex__(tuple(rgb))
        self.solved = True

    def get_solution(self):
        assert self.solved
        return self.V, self.U
    
    def __cost_if__(self, fclass_show, coach_show):
        """
        Calculate the cost if exactly
            - `fclass_show` number of people show up for the first class
            - `coach_show` number of people show up for the Coach class
        """
        cost = 0
        f_class_free_seats = self.n_seats_fclass - fclass_show
        assert f_class_free_seats >= 0

        stipend_customers = max(0, coach_show - self.n_seats_coach)
        assert stipend_customers >= 0

        if stipend_customers == 0:
            return 0
        if stipend_customers < f_class_free_seats:
            return stipend_customers*self.cost_of_bumping
        return f_class_free_seats*self.cost_of_bumping + (stipend_customers-f_class_free_seats)*self.cost_of_stipend

    def __expected_cost__(self, fclass_booked, coach_booked):
        """
        Calculate the expected cost if
            - `fclass_booked` number of people have booked the First class ticket
            - `coach_booked` number of people have booked the Coach ticket
        """
        cost = 0
        bin_prob_fclass = binom(fclass_booked, self.proba_fclass_showing_up)
        bin_prob_coach = binom(coach_booked, self.proba_coach_showing_up)
        for f in range(fclass_booked+1):
            for c in range(coach_booked+1):
                cost += bin_prob_coach.pmf(c)*bin_prob_fclass.pmf(f)*self.__cost_if__(f, c)
        return cost
    
    def __rgb_to_hex__(self, rgb):
        return '#%02x%02x%02x80' % rgb

In [ ]:
ft = FlightTicket()

param_template = ft.get_parameter_template()
for param in param_template.keys():
    ip = input(f"{param_template[param]['prompt']}: ")
    ip = param_template[param]['dtype'](ip)
    param_template[param] = ip

param_template

In [ ]:
ft.set_parameters(param_template)

In [ ]:
ft.calculate_initial_conditions()

In [ ]:
ft.calculate_values()

In [ ]:
ft.V[-1, -1, 0]

In [ ]:
ft.V[:,:,-1].min()

## Manual

In [ ]:
price_coach = [300, 350]                                    # Pc
price_fclass = [425, 500]                                   # Pf

prob_coach = [0.65, 0.30]                                   # Prc
prob_fclass = [0.08, 0.04]                                  # Prf
prob_increase = 0.03

cost_bump = 50                                              # Cb
cost_stipend = 425                                          # Cs
discount = 0.17

prob_show_coach = 0.95
prob_show_fclass = 0.97

seats_coach = 100
seats_fclass = 20

oversell = 5

days_left = 365

In [ ]:
delta = 1/(1+(discount/365))                                # d
prob_coach_fsold = [x + prob_increase for x in prob_coach]     # Prc*
seats_coach_eff = seats_coach + oversell

In [ ]:
s1Values = np.arange(seats_fclass+1)
s2Values = np.arange(seats_coach_eff+1)
tValues = np.arange(days_left+1)
s1N = len(s1Values)
s2N = len(s2Values)
tN = len(tValues)

In [ ]:
V = np.zeros((s1N, s2N, tN))
U = np.array([[['#00000080']*tN]*s2N]*s1N)

In [ ]:
def cost_if(fclass_show, coach_show):
    """
    Calculate the cost if exactly
        - `fclass_show` number of people show up for the first class
        - `coach_show` number of people show up for the Coach class
    """
    cost = 0
    f_class_free_seats = seats_fclass - fclass_show
    assert f_class_free_seats >= 0

    trouble_customers = max(0, coach_show - seats_coach)
    assert trouble_customers >= 0

    if trouble_customers == 0:
        return 0
    if trouble_customers < f_class_free_seats:
        return trouble_customers*cost_bump
    return f_class_free_seats*cost_bump + (trouble_customers-f_class_free_seats)*cost_stipend

def expected_cost(fclass_booked, coach_booked):
    """
    Calculate the expected cost if
        - `fclass_booked` number of people have booked the First class ticket
        - `coach_booked` number of people have booked the Coach ticket
    """
    cost = 0
    bin_prob_fclass = binom(fclass_booked, prob_show_fclass)
    bin_prob_coach = binom(coach_booked, prob_show_coach)
    for f in range(fclass_booked+1):
        for c in range(coach_booked+1):
            cost += bin_prob_coach.pmf(c)*bin_prob_fclass.pmf(f)*cost_if(f, c)
    return cost

In [ ]:
expected_cost(20, 101)

In [ ]:
for s1 in range(s1N):
    for s2 in range(s2N):
        V[s1, s2, tN-1] = -expected_cost(seats_fclass-s1,seats_coach_eff-s2)

In [ ]:
np.equal(V, ft.V)

In [ ]:
def rgb_to_hex(rgb):
    return '#%02x%02x%02x80' % rgb

In [ ]:
for t in reversed(range(tN-1)):
    for s1 in range(s1N):
        for s2 in range(s2N):
            if s1 == 0 and s2 == 0:
                V[s1, s2, t] = delta*V[s1, s2, t+1]
            else:
                prob_coach_eff = prob_coach if s1 != 0 else prob_coach_fsold
                prob_fclass_eff = prob_fclass if s1 != 0 else [0, 0]
                prob_coach_eff = prob_coach_eff if s2 != 0 else [0, 0]
                
                values = list()
                for x1 in range(1,-1,-1):
                    for x2 in range(1, -1, -1):
                        value = price_coach[x1]*prob_coach_eff[x1] + price_fclass[x2]*prob_fclass_eff[x2] + delta*(
                            prob_coach_eff[x1]*prob_fclass_eff[x2]*V[max(s1-1, 0), max(s2-1,0), t+1] + \
                            prob_coach_eff[x1]*(1-prob_fclass_eff[x2])*V[s1, max(s2-1,0), t+1] + \
                            (1-prob_coach_eff[x1])*prob_fclass_eff[x2]*V[max(s1-1, 0), s2, t+1] + \
                            (1-prob_coach_eff[x1])*(1-prob_fclass_eff[x2])*V[s1, s2, t+1]
                        )
                        values.append(value)

                # values[0] -> Coach High,  Fclass High
                # values[1] -> Coach High,  Fclass Low
                # values[2] -> Coach Low,   Fclass High
                # values[3] -> Coach Low,   Fclass Low

                V[s1, s2, t] = max(values)
                tmp = np.argmax(values)

                rgb = [0,0,0]

                # Set the green pixel intensity based on Fclass price - 255 for high price, 127 for low price
                if s1:
                    rgb[1] = ((int(tmp == 0 or tmp == 2) + 1)*128)-1
                
                # Set the blue pixel intensity based on coach price - 255 for high price, 127 for low price
                if s2:
                    rgb[2] = ((int(tmp == 0 or tmp == 1) + 1)*128)-1

                U[s1, s2, t] = rgb_to_hex(tuple(rgb))

In [ ]:
V[-1, -1, 0]

### Plotting 3d

In [ ]:
def make_ax(grid=False):
    fig = plt.figure()
    ax = fig.gca(projection='3d', adjustable='box')
    ax.set_xlabel("FC_sold")
    ax.set_ylabel("Coach_sold")
    ax.set_zlabel("Time")
    ax.grid(grid)
    return ax

def set_axes_equal(ax: plt.Axes):
    limits = np.array([
        ax.get_xlim3d(),
        ax.get_ylim3d(),
        ax.get_zlim3d(),
    ])
    origin = np.mean(limits, axis=1)
    radius = 0.5 * np.max(np.abs(limits[:, 1] - limits[:, 0]))
    _set_axes_radius(ax, origin, radius)

def _set_axes_radius(ax, origin, radius):
    x, y, z = origin
    ax.set_xlim3d([x - radius, x + radius])
    ax.set_ylim3d([y - radius, y + radius])
    ax.set_zlim3d([z - radius, z + radius])

In [ ]:
ax = make_ax(True)
ax.voxels(np.ones(U.shape), facecolors=U, shade=False)
# ax.set_box_aspect([1,1,1])
# set_axes_equal(ax)
plt.show()